In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
playstore = pd.read_csv("https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore.csv")
reviews = pd.read_csv("https://raw.githubusercontent.com/gdv/foundationsCS-2018/master/ex-data/project/googleplaystore_user_reviews.csv")

In [3]:
# punto 1
play = pd.DataFrame(playstore)

In [4]:
play[['firstcol', 'secondcol']] = play['Size'].str.extract('(\d+\.?\d*)([A-Za-z]*)', expand = True)
Size = pd.Series(play['Size'])

In [5]:
numero = pd.Series(play['firstcol'])
valore = pd.Series(play['secondcol'])
a = 0
b = 0

In [ ]:
for i in range(0,10841):
    b = 0
    for j in range(0,10841):
        if a == b:
            if valore[j] == 'M':
                Size[a] = float(numero[i])*1000000
            if valore[j] == 'k':
                Size[a] = float(numero[i])*1000
            if valore[j] == 'G':
                Size[a] = float(numero[i])*1000000000
        b = b+1
    a = a+1

In [ ]:
play['Size'] = Size

In [ ]:
#punto 2 convertire il numero di installazioni in numero

In [ ]:
inst = pd.Series(play['Installs'])

In [ ]:
for i in range(0,len(play.Installs)):
    inst[i] = re.sub('[+,]','',play.Installs[i])

In [ ]:
play['Installs'] = inst

In [ ]:
#punto 3

In [ ]:
play = play.replace('Varies with device', play.replace(['Varies with device'], [None]))

In [ ]:
#punto 4

In [ ]:
cur_ver=pd.Series(play['Current Ver'])
and_ver=pd.Series(play['Android Ver'])
cur_ver_1=pd.Series([''])
and_ver_1=pd.Series([''],index=[0])

In [ ]:
x = ""
for i in cur_ver:
    try:
        x = re.compile('\d+[\d\.]+\d').match(str(i)).group()
        cur_ver_1.set_value(max(cur_ver_1.index) + 1,  x)

    except:
        x = np.nan
        cur_ver_1.set_value(max(cur_ver_1.index) + 1,  x)

In [ ]:
cur_ver_1.value_counts()

In [ ]:
x = ""
for i in and_ver:
    try:
        x = re.compile('\d+[\d\.]+\d').match(str(i)).group()
        and_ver_1.set_value(max(and_ver_1.index) + 1,  x)

    except:
        x = np.nan
        and_ver_1.set_value(max(and_ver_1.index) + 1,  x)

In [ ]:
and_ver_1

In [ ]:
#punto 5 rimuovere i duplicati

In [ ]:
#ipotizzo che i duplicati siano solo del nome

play = play.drop(play[play.Rating > 5].index)
play = play.drop_duplicates(subset=['App'], keep = 'first', inplace = False).sort_index().reset_index(drop=True)

In [ ]:
play['App'].nunique() #mi conta i valori unici che ho per ogni colonna del mio dataframe (play)

In [ ]:
lenght = len(play['App'])
lenght
#il numero di elementi unici e quelli presenti nel dataframe "clean" sono uguali, quindi i duplicati sono stati rimossi.

In [ ]:
# punto 6 per ogni categoria calcola il numero di app

In [ ]:
sub_play = play[['App','Category']]
sub_group = sub_play.groupby('Category').count()
sub_group

In [ ]:
# punto 7 per ogni categoria fare average rating

In [ ]:
play_rat = play[['Rating','Category']]
play_rat = play_rat.groupby('Category').mean()
play_rat

In [ ]:
#punto 8 Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [ ]:
def uniq(input):
    output = []
    for x in input:
        if x not in output:
            output.append(x)
    return output

In [ ]:
genlist = uniq(play['Genres'])
genlistdf = pd.DataFrame(genlist)

In [ ]:
split = genlistdf[0].str.split(";", n = 1, expand = True) 
lista = uniq(split[0])

In [ ]:
def uniq2(input,lista):
    output = lista
    for x in input:
        if x not in output:
            output.append(x)
    return output

In [ ]:
complete = uniq2(split[1],lista)
completedf = pd.DataFrame(complete)
Genere = completedf.dropna()

In [ ]:
App = play[['App','Genres']]
supp = App['Genres'].str.split(';', n = 1, expand = True)
App['first'] = supp[0]
App['second'] = supp[1]
App.drop(columns = ['Genres'], inplace = True) #elimino la colonna

In [ ]:
parte1 = App[['App','first']]
parte2 = App[['App','second']]
parte2 = parte2.dropna()
parte2.rename(columns={'second': 'first'}, inplace=True)
frames = [parte1,parte2]
Genres_App = pd.concat(frames)
Genres_App = Genres_App.sort_values(['App'])
Genres_App.rename(columns={'first': 'Genre'}, inplace=True)

In [ ]:
#punto 9 

In [ ]:
for i in Genere[0]:
    play[i] = False
play[['g1','g2']] = play['Genres'].str.split(';',expand=True)
clean1 = play

In [ ]:
for i in Genere[0]:
    clean1[i].loc[clean1["g1"]==i] = True
for i in Genere[0]:
    clean1[i].loc[clean1["g2"]==i] = True

In [ ]:
#clean1["Creativity"].value_counts().plot("pie")

In [ ]:
#clean1["Art & Design"].value_counts().plot("pie")

In [ ]:
#punto10 For each genre, compute the average rating. What is the genre with highest average?

In [ ]:
clean2= clean1
Genere.join(clean2["Rating"], sort = False)[["Rating"]].groupby(Genere[0]).mean().sort_values(by = 'Rating', ascending = False)

In [ ]:
#punto11 For each app, compute the approximate income, obtain as a product of number of installs and price.

In [ ]:
play['Price'].value_counts().sort_index

In [ ]:
revenue = []
play['Price'].replace('Free',0)
inst = play['Installs']
price = play['Price']
lenght = len(play['App'])

In [ ]:
#for i in range(0,lenght):
#    revenue[i] = int(inst[i]) * float(price[i].replace('$',''))

inst1 = pd.Series(inst).replace('Free',0)
price1 = pd.Series(price).replace('Free',0)
price1 = pd.Series(price1).replace('Everyone',0)

In [ ]:
for i in range(0,lenght):
    revenue.append(int(inst1[i])*(float(price1[i].replace('$',''))))
revenue

In [ ]:
#punto12 For each app, compute its minimum and maximum Sentiment_polarity
reviews = reviews.dropna().sort_index().reset_index(drop=True)

In [ ]:
#massimo
massimo = reviews.groupby(['App']).max().sort_values(['Sentiment_Polarity'], ascending = False)

In [ ]:
#minimo
minimo = reviews.groupby(['App']).min().sort_values(['Sentiment_Polarity'],ascending = True)